In [2]:
import numpy as np
import tensorflow.compat.v1 as tf

In [3]:
tf.disable_eager_execution()

In [6]:
NUM_CLASSES = 20

In [7]:
class MLP:
    def __init__(self, vocab_size, hidden_size):
        self._vocab_size = vocab_size
        self._hidden_size = hidden_size
        self.X = tf.placeholder(tf.float32, shape=[None, vocab_size])
        self._real_Y = tf.placeholder(tf.int32, shape=[None, ])
    
    def build_graph(self):
        w1 = tf.get_variable(
            name = 'weights_input_hidden',
            shape = (self._vocab_size, self._hidden_size),
            initializer = tf.random_normal_initializer(seed=2608)
        )
        b1 = tf.get_variable(
            name = 'biases_input_hidden',
            shape = (self._hidden_size),
            initializer = tf.random_normal_initializer(seed=2608)
        )
        w2 = tf.get_variable(
            name = 'weights_hidden_output',
            shape = (self._hidden_size, NUM_CLASSES),
            initializer = tf.random_normal_initializer(seed=2608)
        )
        b2 = tf.get_variable(
            name = 'biases_hidden_output',
            shape = (NUM_CLASSES),
            initializer = tf.random_normal_initializer(seed=2608)
        )
        
        hidden = tf.matmul(self.X, w1) + b1   # linear hidden layer 
        hidden = tf.sigmoid(hidden)           # activated output
        logits = tf.matmul(hidden, w2) + b2   # linear 
        
        labels_one_hot = tf.one_hot(indices=self._real_Y, depth=NUM_CLASSES, dtype=tf.float32)   # create one hot vector representing ground truth
        loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels_one_hot, logits=logits)   # NOTICE: CRE loss working with logits input
        loss = tf.reduce_mean(loss)
        
        probs = tf.nn.softmax(logits)
        predicted_labels = tf.argmax(probs, axis=1)
        predicted_labels = tf.squeeze(predicted_labels)
        
        return predicted_labels, loss
    
    def trainer(self, loss, lr):
        optimizer = tf.train.AdamOptimizer(lr).minimize(loss)
        return optimizer
        
    

In [8]:
import random

class DataReader:
    def __init__(self, data_path, batch_size, vocab_size):
        
        self._batch_size = batch_size
        
        with open(data_path) as f:
            doc_lines = f.read().splitlines()
        
        self._data  = []
        self._labels = []
        
        for data_id, line in enumerate(doc_lines):
            vector = [0.0 for _ in range(vocab_size)]
            features = line.split('<fff>')
            label, doc_id = int(features[0]), int(features[1])
            tfidfs = features[2].split()
            for tfidf in tfidfs:
                idx = int(tfidf.split(':')[0])
                val = float(tfidf.split(':')[1])
                vector[idx] = val
            self._data.append(vector)
            self._labels.append(label)
        
        self._data = np.array(self._data)
        self._labels = np.array(self._labels)
        
        self._num_epochs = 0
        self._batch_id = 0
        
    def next_batch(self):
        start = self._batch_id * self._batch_size   # starting idx of the batch
        end = start + self._batch_size              # ending idx of the batch
        self._batch_id += 1                         
        
        if end + self._batch_size > len(self._data):    # when the last batch exceed the length of dataset
            end = len(self._data)
            self._num_epochs += 1
            self._batch_id = 0                         # reset _batch_id = 0 for the next epoch
            indices = list(range(len(self._data)))
            random.seed(2608)
            random.shuffle(indices)                    # shuffle data for the next epoch
            self._data = self._data[indices]
            self._labels = self._labels[indices]
        
        return self._data[start : end], self._labels[start : end]

In [9]:
def save_params(name, val, epoch):
    file_name = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
    if len(val.shape) == 1:   # biases
        string_form = ','.join([str(num) for num in val])
    else:    # weights matrix
        string_form = '\n'.join([','.join([str(num) for num in val[row]])
                                for row in range(val.shape[0])])
    # with open('D:\\movedFromC\\123\\LabML\\session1\\MLLab\\session3\\params\\' + filename, 'w') as f:    # run on local
    #     f.write(string_form)

    with open('/content/drive/My Drive/Colab Notebooks/LAB/session3/params/' + file_name, 'w') as f:    # run on local
      f.write(string_form)

In [10]:
def restore_params(name, epoch):
    file_name = name.replace(':', '-colon-') + '-epoch-{}.txt'.format(epoch)
    with open('/content/drive/My Drive/Colab Notebooks/LAB/session3/params/' + file_name) as f:
        lines = f.readlines()
    if len(lines) == 1:   # biases
        val = [float(num) for num in lines[0].split(',')]
    else:    # weights matrix
        val = [[float(num) for num in lines[row].split(',')]
                  for row in range(len(lines))]
    return val
    

In [11]:
import matplotlib.pyplot as plt

def train():
    
  with open('/content/drive/My Drive/Colab Notebooks/PRJ2/words_idfs.txt') as f:
    vocab_size = len(f.readlines())
    
  def load_dataset():
    train_reader = DataReader(
        data_path = '/content/drive/My Drive/Colab Notebooks/PRJ2/words_tfidfs.txt',
        batch_size = 32,
        vocab_size = vocab_size
    )
    return train_reader
    
  mlp = MLP(vocab_size = vocab_size, hidden_size = 50)

  predicted_labels, loss = mlp.build_graph()
  train_op = mlp.trainer(loss, lr = 0.001)

  train_loss = []   # for plotting
  steps = []
    
  with tf.Session() as sess:
    train_reader = load_dataset()
    epoch, MAX_EPOCHS = 0, 10000

    sess.run(tf.global_variables_initializer())

    while epoch < MAX_EPOCHS:
      train_data, train_labels = train_reader.next_batch()
      p_labels_eval, loss_eval, _ = sess.run(
        [predicted_labels, loss, train_op],
        feed_dict={
            mlp.X: train_data,
            mlp._real_Y: train_labels
        }
      )
      epoch += 1
      # train_loss.append(loss_eval)
      # steps.append(epoch)
      if epoch % 500 == 0:
        print ("step: {}, loss: {}".format(epoch, loss_eval))
      if loss_eval < 1e-5:
        break
    trainable_variables = tf.trainable_variables()
    for var in trainable_variables:
      save_params(name=var.name, val=var.eval(), epoch=train_reader._num_epochs)

  return train_reader._num_epochs   # return the number of epochs to pass to the test funtion      
        # plt.plot(steps[:1000], train_loss[:1000], 'dt')
        # plt.xlabel('Epoch')
        # plt.ylabel('Loss')
        # plt.show()

In [12]:
def test(num_epochs):
    with open('/content/drive/My Drive/Colab Notebooks/PRJ2/words_idfs.txt') as f:
        vocab_size = len(f.readlines())

    test_reader = DataReader(
        data_path = '/content/drive/My Drive/Colab Notebooks/PRJ2/words_tfidfs_test.txt',
        batch_size = 32,
        vocab_size = vocab_size
    )
    
    mlp = MLP(vocab_size = vocab_size, hidden_size = 50)
    predicted_labels,loss = mlp.build_graph()
    
    with tf.Session() as sess:
        epoch = num_epochs
        
        trainable_variables = tf.trainable_variables()
        for var in trainable_variables:
            saved_val = restore_params(var.name, epoch)
            assign_op = var.assign(saved_val)
            sess.run(assign_op)
        corr_pred = 0
        while True:
            test_data, test_labels = test_reader.next_batch()
            test_plabels_eval = sess.run(predicted_labels,
                                        feed_dict={
                                            mlp.X: test_data,
                                            mlp._real_Y: test_labels
                                            }
                                        )
            matches = np.equal(test_plabels_eval, test_labels)
            corr_pred += np.sum(matches.astype(float))
            if test_reader._batch_id == 0: 
                break
        print('Epoch: ', epoch)
        print('Acc: ', corr_pred/len(test_reader._data))

In [ ]:
# Firstly, train the model to get the number of epochs, then pass that number
# as a param to test function

tf.reset_default_graph()
epoch = train()

In [13]:
tf.reset_default_graph()
test(epoch)